In [ ]:
data_path = 'internship_data'

In [ ]:
import torch
import numpy as np
import tqdm


np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True


Определяем аугментации для тренировочной и валидационной выборки

In [ ]:
from torchvision import transforms

input_size = 224
train_transforms = transforms.Compose([
                                       transforms.Resize((input_size,input_size)),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomRotation(45),
                                       transforms.RandomGrayscale(p=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406], 
                                           [0.229, 0.224, 0.225])
])
valid_transforms = transforms.Compose([
                                       transforms.Resize((input_size, input_size)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406], 
                                           [0.229, 0.224, 0.225])
])

In [ ]:

valid_size = 0.2
batch_size = 16

Разделяем датасет на тренировочную и валидационную выборки. Определяем соответствующие даталоадеры

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import SubsetRandomSampler, DataLoader
train_dataset = ImageFolder(data_path, train_transforms)
valid_dataset = ImageFolder(data_path, valid_transforms)

num_train = len(train_dataset) 
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler = train_sampler)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, sampler = valid_sampler)

Определяем функции для обучения сети и валидиции

In [ ]:
def train(model, loader, loss_fn, optimizer, device, scheduler=None):
    model.train()
    train_loss = []
    train_acc = []

    for images, labels in tqdm.tqdm(loader, total=len(loader), desc="training...", position=0, leave=True):
        images = images.to(device)  
        labels = labels.to(device) 

        preds = model(images) 
        loss = loss_fn(preds, labels)
        preds_class = preds.argmax(dim=1)

        train_loss.append(loss.item())
        train_acc.append((preds_class == labels.data).float().mean())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if scheduler:
        scheduler.step()

    return np.mean(train_loss, dtype=np.float64), np.mean(train_acc, dtype=np.float64)


def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    val_acc = []
    for images, labels in tqdm.tqdm(loader, total=len(loader), desc="validation...", position=0, leave=True):
        images = images.to(device)  
        labels = labels.to(device)

        with torch.no_grad():
            preds = model(images) 
        loss = loss_fn(preds, labels)
        preds_class = preds.argmax(dim=1)

        val_loss.append(loss.item())
        val_acc.append((preds_class == labels.data).float().mean())

    return np.mean(val_loss, dtype=np.float64), np.mean(val_acc, dtype=np.float64)



Определяем архитектуру сети, функцию потерь и метод оптимизации

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models


device = "cuda" if torch.cuda.is_available() else "cpu"
LR = 1e-3

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=LR, amsgrad=True)
loss_fn = nn.CrossEntropyLoss()

Выполняем обучение. При улучшении лосса на валидации на текущей эпохе сохраняем модель

In [ ]:
num_epochs = 15
best_val_loss = np.inf
for epoch in range(num_epochs):

    train_loss, train_acc = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss, val_acc = validate(model, valid_dataloader, loss_fn, device=device)
    

    print("Epoch #{:2}:\ttrain loss: {:10.7}\tval loss: {:10.7}\ttrain acc: {:10.7}\tval acc: {:10.7}".format(epoch, train_loss, val_loss, train_acc, val_acc))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"model.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)